In [9]:
import sys
sys.path.insert(0, "../../../pymc3")

import numpy as np
import pymc3 as pm
import arviz as az

from aesara import tensor as at
import aesara

In [67]:
X = np.array([-1, 0.5, 1])
K = 20

def stick_breaking(betas):
    portion_remaining = at.concatenate([[1], (1 - betas[:-1])])
    
    return at.mul(betas, at.cumprod(portion_remaining))

### Testing GP submodule

Sampling from a GP prior

In [3]:
with pm.Model() as model:
    gp = pm.gp.Latent()
    gp.prior("gp-prior", X)
    trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [gp-prior_rotated_]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 18 seconds.


### Trying to sample from a DP Prior

In [55]:
a = np.array([10., 5, 1, 4])
a = np.atleast_2d(a)

b = np.array([1, 2, 3])
b = b[..., np.newaxis]

In [100]:
X = np.array([-1, 0.5, 1])
X = X[..., np.newaxis]
X

array([[-1. ],
       [ 0.5],
       [ 1. ]])

In [114]:
with pm.Model() as model:
    alpha = pm.Gamma("alpha", 1, 1, shape=1)
#     betas = pm.Beta("betas", 1, alpha, shape=K)
#     w = pm.Deterministic("w", stick_breaking(betas))
    w = pm.Dirichlet("w", a=alpha, shape=K)
    
    base_dist = pm.Normal("G0", mu=0, sigma=1, shape=(1, K))
    
    dp = pm.Deterministic(
        name="dp", 
        var=at.sum(at.mul(w, at.gt(X, base_dist)), axis=1),
    )
    
    trace = pm.sample(
        draws=1000,
        chains=1,
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, w, G0]


Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 10 seconds.
There were 851 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.8847542943133789, but should be close to 0.8. Try to increase the number of tuning steps.
Only one chain was sampled, this makes it impossible to run some convergence checks


In [113]:
trace.to_dict()["posterior"]["dp"][0].mean(axis=0)

/Users/larryshamalama/anaconda3/envs/pymc3-dev-py39/lib/python3.9/site-packages/arviz/data/inference_data.py:436: UserWarning: The attributes are not same for all groups. Considering only the first group `attrs`
  warnings.warn(


array([0.15939608, 0.68399616, 0.84316522])

In [104]:
with pm.Model() as model:
    base_dist = pm.Normal("G0", 0, 1, shape=(K,))
    dp = pm.dp.DirichletProcess(
        name="dp",
        alpha=1,
        base_dist=base_dist,
        K=K,
    )
    dp.prior("prior", X)
    
    trace = pm.sample(
        draws=1000,
        chains=1,
    )

AttributeError: module 'pymc3' has no attribute 'dp'

In [116]:
at.newaxis

AttributeError: module 'aesara.tensor' has no attribute 'newaxis'